# SQL Practice 11/20/2022

## Q1

New users sign up with emails and each recieve a text confirmation to activate their account. Given the tables below, write a query to display the id of users who did not confirm on the first day of sign up, but confirmed on the second day.

`emails` **Table**

Column Name | Type
------------|-----
email_id | integer
user_id | integer
signup_date | datetime

`texts` **Table**

Column Name | Type
------------|------
text_id | integer
email_id | integer
signup_action | string
action_date | datetime

`signup_action` can only take the values of `"Confirmed"`, or `"Not Confirmed"`. `action_date` is the date the user confirmed their sign up using the text.

My solution is below

```SQL
SELECT emails.user_id
  FROM emails LEFT JOIN texts
    ON emails.email_id = texts.email_id
    
  WHERE EXTRACT(DAYS FROM (texts.action_date - emails.signup_date)) = 1  
    AND texts.signup_action = 'Confirmed'
;
```

We need to join the two tables using the `email_id` as the common key. Now we need a way to filter by those who confirmed on the next day, we can take the difference of the two dates and use the `EXTRACT()` function to only look at the number of days.

This can also be done by using the `INTERVAL` statement to add a day to the `signup_date` and compare that to the `action_date`. That query is below.

```SQL
SELECT emails.user_id
  FROM emails LEFT JOIN texts
    ON emails.email_id = texts.email_id
    
  WHERE texts.action_date = emails.email_date + INTERVAL '1 DAY'  
    AND texts.signup_action = 'Confirmed'
;
```

## Q2

Your company wants to launch a new credit card, you are asked to analyze how many new credit cards were issued each month. Given the table below, write a query that outputs the name of each credit card and the difference in issued amount between the month with the most cards issued and the month with the least cards issued. Order the results according to the biggest difference.

Data Information

`monthly_cards_issued` **Table**

Column Name | Type
------------|------
issue_month | integer
issue_year | integer
card_name | string
issued_amount | integer

My solution is below

```SQL
SELECT card_name,
    MAX(issued_amount) - MIN(issued_amount) AS difference 
  FROM monthly_cards_issued
  
  GROUP BY card_name
  
  ORDER BY difference DESC
;
```

## Q3

You are trying to find the mean number of items bought per order on an online store, rounded to 1 decimal place.

Data Information

`items_per_order` **Table** 

Column Name | Type
------------|------
item_count | integer
order_occurences | integer

So, we are given the number of orders that have a specific number of items in them, but not the order data itself.

My solution is below

```SQL
SELECT ROUND(SUM(item_count::DECIMAL * order_occurrences) / SUM(order_occurrences)
      , 1) AS mean
  FROM items_per_order;
```

Given the information that we have we just need to compute a weighted average. Lastly, we need to force the output to be a float instead of an integer, so we convert one of the columns in the calculation to a `DECIMAL` type using the `::` operator. 

## Q4

A pharmacy wants to better understand how different drugs are selling. Given the data below, write a query to find the top most profitable drugs sold and how much profit they made. 

Data Information

`pharmacy_sales` **Table**

Column Name | Type
------------|------
product_id | integer
units_sold | integer
total_sales | decimal
cogs | decimal
manufactuter | varchar
drug | varchar

`cogs` stands for "Cost of Goods Sold," which is the direct cost associated with producing the drug. `total_sales` is the revenue generated by all sales of the drug. You may assume there are no ties in profits and that no product has more than one manufacturer.

My solution is below

```SQL
SELECT drug,
    total_sales - cogs AS profit

  FROM pharmacy_sales
  
  ORDER BY profit DESC

  LIMIT 3
;
```